### Scrape Schedule

In [17]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Scrape the NFL schedule for the current season
url = "https://www.pro-football-reference.com/years/2025/games.htm"
response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

# Find the schedule table
table = soup.find("table", id="games")

# Extract headers
headers = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]

# Extract rows
rows = []
for row in table.find("tbody").find_all("tr"):
    if "class" in row.attrs and "thead" in row["class"]:
        continue  # skip mid-table headers
    cells = [cell.get_text(strip=True) for cell in row.find_all(["th", "td"])]
    rows.append(cells)

# Create DataFrame
sked_df = pd.DataFrame(rows, columns=headers)
sked_df.columns = ['Week', 'Day', 'Date', 'Time', 'Visitor', 'H/A', 'Home', 'Boxscore', 'PtsW', 'PtsL', 'YdsW', 'TOW', 'YdsL', 'TOL']
sked_df = sked_df[['Week', 'Day', 'Date', 'Time', 'Visitor', 'H/A', 'Home']]
sked_df['Matchup String'] = sked_df['Visitor'] + " " + sked_df['H/A'].apply(lambda x: "vs" if x == "" else "@") + " " + sked_df['Home']
sked_df['Start Time'] = sked_df['Day'] + ' ' + sked_df['Time']
# roll up the start time column into buckets: Thursday, Sunday Morning, Sunday Early Afternoon, Sunday Late Afternoon, Sunday Night, Monday Night... I don't care about Friday or Saturday
def bucket_start_time(start_time):
    if "Thu" in start_time:
        return "Thursday"
    elif "Mon" in start_time:
        return "Monday Night"
    elif "Sun" in start_time:
        time_part = start_time.split(' ')[-1]
        hour = int(time_part.split(':')[0])
        if hour < 12 and time_part.endswith('AM'):
            return "Sunday Morning"
        elif 1 <= hour < 4 and time_part.endswith('PM'):
            return "Sunday Early Afternoon"
        elif 4 <= hour < 8 and time_part.endswith('PM'):
            return "Sunday Late Afternoon"
        else:
            return "Sunday Night"
    else:
        return "Other"
sked_df['Week'] = sked_df['Week'].astype(int)
sked_df['Start Time Bucket'] = sked_df['Start Time'].apply(bucket_start_time)
sked_df

,Week,Day,Date,Time,Visitor,H/A,Home,Matchup String,Start Time,Start Time Bucket
0,1,Thu,2025-09-04,8:20PM,Philadelphia Eagles,,Dallas Cowboys,Philadelphia Eagles vs Dallas Cowboys,Thu 8:20PM,Thursday
1,1,Fri,2025-09-05,8:00PM,Los Angeles Chargers,,Kansas City Chiefs,Los Angeles Chargers vs Kansas City Chiefs,Fri 8:00PM,Other
2,1,Sun,2025-09-07,1:00PM,Pittsburgh Steelers,@,New York Jets,Pittsburgh Steelers @ New York Jets,Sun 1:00PM,Sunday Early Afternoon
3,1,Sun,2025-09-07,1:00PM,Washington Commanders,,New York Giants,Washington Commanders vs New York Giants,Sun 1:00PM,Sunday Early Afternoon
4,1,Sun,2025-09-07,1:00PM,Las Vegas Raiders,@,New England Patriots,Las Vegas Raiders @ New England Patriots,Sun 1:00PM,Sunday Early Afternoon
...,...,...,...,...,...,...,...,...,...,...
267,18,Sun,2026-01-04,1:00PM,Tennessee Titans,@,Jacksonville Jaguars,Tennessee Titans @ Jacksonville Jaguars,Sun 1:00PM,Sunday Early Afternoon
268,18,Sun,2026-01-04,1:00PM,Baltimore Ravens,@,Pittsburgh Steelers,Baltimore Ravens @ Pittsburgh Steelers,Sun 1:00PM,Sunday Early Afternoon
269,18,Sun,2026-01-04,1:00PM,Los Angeles Chargers,@,Denver Broncos,Los Angeles Chargers @ Denver Broncos,Sun 1:00PM,Sunday Early Afternoon
270,18,Sun,2026-01-04,1:00PM,Seattle Seahawks,@,San Francisco 49ers,Seattle Seahawks @ San Francisco 49ers,Sun 1:00PM,Sunday Early Afternoon


In [40]:
pivoted = pd.pivot_table(sked_df, index='Start Time Bucket', columns='Week', values='Matchup String', aggfunc='count').fillna(0).astype(int)
pivoted = pivoted.reindex(index=["Thursday", "Sunday Morning", "Sunday Early Afternoon", "Sunday Late Afternoon", "Sunday Night", "Monday Night", "Other"])
# drop the index labels
pivoted.index.name = None
# add a color gradient to the dataframe
pivoted.style.background_gradient(axis=1)

Week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Thursday,1,1,1,1,1,1,1,1,1,1,1,1,3,1,1,1,3,0
Sunday Morning,0,0,0,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0
Sunday Early Afternoon,8,9,9,7,6,7,6,7,8,7,7,7,7,8,8,7,10,16
Sunday Late Afternoon,4,3,4,4,4,3,4,3,3,3,4,4,3,3,5,4,1,0
Sunday Night,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
Monday Night,1,2,1,2,1,2,2,1,1,1,1,1,1,1,1,1,1,0
Other,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0
